# Captum tests

Needed to modify the model code slightly, so the rest of the repo won't work

Don't forget to download the models from the README

In [ ]:
import argparse
import multiprocessing as mp
import os
import os.path as osp

import numpy as np
import torch
import yaml
from munch import Munch
from softgroup.data import build_dataloader, build_dataset
from softgroup.model import SoftGroup
from softgroup.util import (collect_results_cpu, get_dist_info, get_root_logger, init_dist,
                            is_main_process, load_checkpoint, rle_decode)
from torch.nn.parallel import DistributedDataParallel
from tqdm import tqdm

args = argparse.Namespace(
    config="./configs/softgroup/softgroup_s3dis_fold5.yaml",
    # Download and extract this
    checkpoint="./models/softgroup_s3dis_spconv2.pth",
    dist=False,
)

cfg_txt = open(args.config, 'r').read()
cfg = Munch.fromDict(yaml.safe_load(cfg_txt))
logger = get_root_logger()

model = SoftGroup(**cfg.model).cuda()
logger.info(f'Load state dict from {args.checkpoint}')
load_checkpoint(args.checkpoint, logger, model)

dataset = build_dataset(cfg.data.test, logger)
dataloader = build_dataloader(dataset, training=False, dist=args.dist, **cfg.dataloader.test)


In [ ]:
with torch.no_grad():
    model.eval()
    for i, batch in tqdm(enumerate(dataloader), total=len(dataset)):
        # Set params here so the model only takes coords and feats as input
        #model.set_params(**batch)
        #result = model(batch['coords_float'], batch['feats'])
        #results.append(result)
        #print(batch[ "scan_ids" ])
        # selected rooms: 4
        if i + 1 == 4:
            break

Input tensors

In [ ]:
print(batch.keys())
#print(batch['voxel_coords'].shape)
#print(batch['coords_float'].shape)
#print(batch['feats'].shape)
#batch['scan_ids']
#batch['batch_idxs']
#batch['voxel_coords']
#batch['p2v_map']
#batch['v2p_map']
#print(batch['coords_float'])
#batch['feats']
#batch['semantic_labels']
#batch['instance_labels']
#batch['instance_pointnum']
#batch['instance_cls']
#batch['pt_offset_labels']
#batch['spatial_shape']
#batch['batch_size']

Show point cloud

In [ ]:
import nubilum.utils.nubilum_utils as nb_utils
def show_color_cloud(batch):
    colors = (batch['feats'] + 1)/2*255 # change color interval from [-1, 1] to [0, 255]
    nb_utils.show_point_cloud(batch["coords_float"], colors)

In [ ]:
show_color_cloud(batch)

# Outputs

    0: 'ceiling',
    1: 'floor',
    2: 'wall',
    3: 'beam',
    4: 'column',
    5: 'window',
    6: 'door',
    7: 'chair',
    8: 'table',
    9: 'bookcase',
    10: 'sofa',
    11: 'board',
    12: 'clutter',

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

model.train()
model.set_params(**batch)

# output always have this format: dict_keys(['scan_id', 'semantic_labels', 'instance_labels', 'coords_float', 'color_feats', 'semantic_preds', 'offset_preds', 'offset_labels', 'semantic_scores'])
# remember that the order of points in the output is different from the batch

out = model(batch['coords_float'], batch['feats'])

instance_labels = torch.LongTensor(out['instance_labels'])

classes = {0: 'ceiling', 1: 'floor', 2: 'wall', 3: 'beam', 4: 'column', 5: 'window', 6: 'door', 7: 'chair', 8: 'table', 9: 'bookcase', 10: 'sofa', 11: 'board', 12: 'clutter'}

nb_utils.show_point_cloud_classification_plotly(out['coords_float'], out['semantic_preds'], instance_labels, classes_dict=classes)
#nb_utils.show_point_cloud_classification_k3d(batch['coords_float'], out['semantic_preds'])

In [ ]:
import matplotlib.pyplot as plt

#model.train()
#model.set_params(**batch)
out = model(batch['coords_float'], batch['feats'])
#print(out[:10])
# N = number of points
# K = number of classes
print(out.keys())
print(out['semantic_scores'].shape) # N x K # out is the probabilities for each possible target. The sum of all prob. is 1.

# Chose a random point of interest
poi = 145 # [index, class]
confidences = out[poi].cpu().numpy()
plt.imshow(confidences.reshape(-1,1))
plt.title(f"Confidences for point of interest {poi}")
confidence, classification = out[poi].max(0)
print(confidence)   # Confidence is the probability of the class target choosed by the model. The biggest prob.
print(classification) # Classification is the class index.
print(classification.shape)

In [ ]:
# Baseline Point Cloud
baseline_coords, baseline_colors = nb_utils.create_baseline_point_cloud(batch["coords_float"])
baseline_coords = baseline_coords.float().to("cuda:0").requires_grad_(True)
baseline_colors = baseline_colors.float().to("cuda:0").requires_grad_(True)
#print(baseline_coords.shape)
#print(baseline_colors.shape)
#nb_utils.show_point_cloud(baseline_coords, baseline_colors)

In [ ]:
# Baseline classification
baseline_out = out = model(baseline_coords, baseline_colors)

instance_labels = torch.LongTensor(baseline_out['instance_labels'])

nb_utils.show_point_cloud_classification_plotly(baseline_out['coords_float'], baseline_out['semantic_preds'], instance_labels, classes_dict=classes)


# Model Wrappers

In [ ]:
from nubilum.forward.nubilum_forward import InstanceWrappedModel, PointWrappedModel, SummarizedWrappedModel 
        
instance_wrap = InstanceWrappedModel(model)
point_wrap = PointWrappedModel(model)
summarized_wrap = SummarizedWrappedModel(model)

In [ ]:
# INTEGRATED GRADIENTS BASED ON OBJECT INSTANCE
from captum.attr import IntegratedGradients
from nubilum.attr.nubilum_integrated_gradients import NubilumIntegratedGradients

model.train() # TODO: See if this works, if not, use model.train
model.set_params(**batch)

coords_float = batch['coords_float'].float().to("cuda:0").requires_grad_(True)
feats = batch['feats'].float().to("cuda:0").requires_grad_(True)

ig_instance = NubilumIntegratedGradients(instance_wrap)
ig_point = NubilumIntegratedGradients(point_wrap)
ig_summary = NubilumIntegratedGradients(summarized_wrap)

In [ ]:
# Point of interest on evidence in the Point Cloud

# REMEMBER TO USE ALWAYS THE POINTS INDICES FROM THE OUTPUT, THANKS SOFTGROUP FOR CHANGING THEM :DD

poi = 243012 # table leg with conflict
ioi = 16
ioi_floor = 39
chairs = torch.tensor(7, device="cuda") # remember to always change this when changing the analysis
tables = torch.tensor(8, device="cuda")
bookcases = torch.tensor(9, device="cuda")
windows = torch.tensor(5, device="cuda")
doors = torch.tensor(6, device="cuda")
floor = torch.tensor(1, device="cuda")
#nb_utils.show_poi(poi, out["coords_float"])

In [ ]:
instance_wrap_attributes = ig_instance.attribute((coords_float, feats), baselines=(baseline_coords, baseline_colors), target=chairs, additional_forward_args=(out['instance_labels'], ioi, 'semantic_scores'), n_steps=50, method='riemann_middle', internal_batch_size=1)
inst_point_attrs = nb_utils.sum_point_attributes(instance_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(inst_point_attrs, batch['coords_float'], "All attributes")

In [ ]:
# Chair importance ratio
instance_mask = batch['instance_labels'] == ioi
chair_attr = inst_point_attrs[instance_mask]

ratio = chair_attr.sum() / inst_point_attrs.sum()
print(ratio)

In [ ]:
floor_ig_attributes = ig_instance.attribute((coords_float, feats), baselines=(baseline_coords, baseline_colors), target=floor, additional_forward_args=(out['instance_labels'], ioi_floor, 'semantic_scores'), n_steps=50, method='riemann_middle', internal_batch_size=1)
floor_inst_point_attrs = nb_utils.sum_point_attributes(floor_ig_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(floor_inst_point_attrs, batch['coords_float'], "All attributes")

In [ ]:
# Chair importance ratio
floor_instance_mask = batch['instance_labels'] == ioi_floor
floor_attr = floor_inst_point_attrs[floor_instance_mask]

floor_ratio = floor_attr.sum() / floor_inst_point_attrs.sum()
print(floor_ratio)

In [ ]:
point_wrap_attributes = ig_point.attribute((coords_float, feats), baselines=(baseline_coords, baseline_colors), target=chairs, additional_forward_args=(poi, 'semantic_scores'), method='riemann_middle', internal_batch_size=1)
point_point_attrs = nb_utils.sum_point_attributes(point_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(point_point_attrs, batch['coords_float'], "All attributes")

In [ ]:
summary_wrap_attributes = ig_summary.attribute((coords_float, feats), baselines=(baseline_coords, baseline_colors), target=tables, additional_forward_args=('semantic_scores'), method='riemann_middle', internal_batch_size=1)
summ_point_attrs = nb_utils.sum_point_attributes(summary_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(summ_point_attrs, batch['coords_float'], "All attributes")

# No Baseline

In [ ]:
instance_wrap_attributes = ig_instance.attribute((coords_float, feats), target=chairs, additional_forward_args=(out['instance_labels'], ioi, 'semantic_scores'), n_steps=50, method='riemann_middle', internal_batch_size=1)
inst_point_attrs = nb_utils.sum_point_attributes(instance_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(inst_point_attrs, batch['coords_float'], "All attributes")

In [ ]:
summary_wrap_attributes = ig_summary.attribute((coords_float, feats), target=tables, additional_forward_args=('semantic_scores'), method='riemann_middle', internal_batch_size=1)
summ_point_attrs = nb_utils.sum_point_attributes(summary_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(summ_point_attrs, batch['coords_float'], "All attributes")